### Run with R_scRNAseq

In [1]:
suppressPackageStartupMessages({library(dplyr)
library(Seurat)
library(readr)
library(SingleCellExperiment, quietly = TRUE)
library(scater, quietly = TRUE)
library(tidyr)
library(purrr)
library(cowplot)
library(ggrepel)
library(viridis)
library(ggdendro)
library(gridExtra)
library(egg)
library(pheatmap)})

Warning message:
"package 'Seurat' was built under R version 3.6.2"Warning message:
"package 'matrixStats' was built under R version 3.6.2"

In [4]:
setwd("/staging/leuven/stg_00041/Adrian/TALON_JANISZEWSKI_XCR2/")

### Read R object 

In [5]:
reprogramming.big <- readRDS("Schiebinger_integration/reprogramming.big.rds")

In [6]:
reprogramming.big

Loading required package: Seurat
Warning message:
"package 'Seurat' was built under R version 3.6.2"Registered S3 method overwritten by 'R.oo':
  method        from       
  throw.default R.methodsS3
Registered S3 methods overwritten by 'ggplot2':
  method         from 
  [.quosures     rlang
  c.quosures     rlang
  print.quosures rlang


An object of class Seurat 
28703 features across 57536 samples within 1 assay 
Active assay: RNA (28703 features)

In [4]:
reprogramming.big

An object of class Seurat 
28703 features across 57536 samples within 1 assay 
Active assay: RNA (28703 features)

In [7]:
reprogramming.big_norm_counts <- reprogramming.big@assays$RNA@data

In [8]:
write.csv(reprogramming.big_norm_counts, 'SCENIC/resources_folder/reprogramming.big_norm_counts.csv')

Extract normalized counts from object from John

In [7]:
DimPlot(reprogramming.big, label = T, label.size = 2) + NoLegend()

ERROR: Error: Unable to find a DimReduc matching one of 'umap', 'tsne', or 'pca', please specify a dimensional reduction to use


# RUN SCENIC 

## pySCENIC2

In [1]:
import os
import glob
import pickle
import pandas as pd
import numpy as np

In [2]:
from dask.diagnostics import ProgressBar

/data/leuven/320/vsc32023/miniconda/envs/pySCENIC_2/lib/python3.6/site-packages/dask/config.py:161: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  data = yaml.load(f.read()) or {}


In [3]:
from arboreto.utils import load_tf_names
from arboreto.algo import grnboost2

In [4]:
from pyscenic.rnkdb import FeatherRankingDatabase as RankingDatabase
from pyscenic.utils import modules_from_adjacencies, load_motifs
from pyscenic.prune import prune2df, df2regulons
from pyscenic.aucell import aucell

In [5]:
import seaborn as sns

In [6]:
cd "/staging/leuven/stg_00041/Adrian/TALON_JANISZEWSKI_XCR2/"

/ddn1/vol1/staging/leuven/stg_00041/Adrian/AJ_0069_Schiebinger_SCENIC


In [7]:
ROOT_FOLDER="/staging/leuven/stg_00041/Adrian/TALON_JANISZEWSKI_XCR2/"

DATA_FOLDER = os.path.join(ROOT_FOLDER,"SCENIC/Schiebinger/data_folder")
RESOURCES_FOLDER = os.path.join(ROOT_FOLDER,"SCENIC/resources_folder")
DATABASE_FOLDER = ("/staging/leuven/stg_00041/Adrian/TALON_JANISZEWSKI_XCR2/SCENIC/database_folder/")
                         
SCHEDULER="123.122.8.24:8786"
                         
DATABASES_GLOB = os.path.join(DATABASE_FOLDER, "mm10*.mc9nr.feather")
MOTIF_ANNOTATIONS_FNAME = os.path.join(RESOURCES_FOLDER, "motifs-v9-nr.mgi-m0.001-o0.0.tbl")
MM_TFS_FNAME = os.path.join(RESOURCES_FOLDER, 'mm_mgi_tfs.txt')

In [8]:
tf_names = load_tf_names(MM_TFS_FNAME)

In [9]:
db_fnames = glob.glob(DATABASES_GLOB)
def name(fname):
    return os.path.splitext(os.path.basename(fname))[0]
dbs = [RankingDatabase(fname=fname, name=name(fname)) for fname in db_fnames]
dbs

[FeatherRankingDatabase(name="mm10__refseq-r80__500bp_up_and_100bp_down_tss.mc9nr"),
 FeatherRankingDatabase(name="mm10__refseq-r80__10kb_up_and_down_tss.mc9nr")]

In [10]:
from distributed import Client, LocalCluster

### RUN SCENIC

In [14]:
SC_EXP_FNAME = os.path.join(RESOURCES_FOLDER, "SCENIC/reprogramming.big_norm_counts.csv")
REGULONS_FNAME = os.path.join(DATA_FOLDER, "reprogramming_big_norm_regulons_1.p")
MOTIFS_FNAME = os.path.join(DATA_FOLDER, "reprogramming_big_norm_motifs_1.csv")

In [15]:
exp_mtx = pd.read_csv(SC_EXP_FNAME, index_col=0, sep=',').T

In [13]:
local_cluster = LocalCluster(n_workers=8, threads_per_worker=1)
client = Client(local_cluster)

In [14]:
exp_mtx.head()

,Sox17,Mrpl15,Lypla1,Tcea1,Rgs20,Atp6v1h,Rb1cc1,4732440D04Rik,Fam150a,Pcmtd1,...,mt-Ta,mt-Tn,mt-Tc,mt-Tr,mt-Ts2,mt-Tl2,mt-Te,mt-Tt,mt-Tp,pCX-eGFP
Schiebinger_DiPSC_2i_C1_CCATGTCAGACTAGGC.1,0.0,0.585802,1.220622,0.952763,0.0,0.000000,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Schiebinger_D8.5_2i_C1_CGGACTGGTAATTGGA.1,0.0,1.324153,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Schiebinger_D4_Dox_C1_AGAGCTTCAGGAATGC.1,0.0,0.511066,0.000000,1.299720,0.0,0.000000,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Schiebinger_D12.5_2i_C1_GTCTTCGGTGACGGTA.1,0.0,1.098301,0.000000,0.510639,0.0,0.510639,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Schiebinger_D3_Dox_C2_TCTGGAAAGCTGCGAA.1,0.0,0.847184,0.510746,0.510746,0.0,0.000000,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
reprogramming_big_norm_adjacencies_1 = grnboost2(
    expression_data=exp_mtx,
    tf_names=tf_names,
    verbose=True,
    client_or_address=client)

preparing dask client
parsing input
creating dask graph


/data/leuven/320/vsc32023/miniconda/envs/pySCENIC_2/lib/python3.6/site-packages/arboreto/algo.py:214: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  expression_matrix = expression_data.as_matrix()


15 partitions
computing dask graph


In [21]:
reprogramming_big_norm_adjacencies_1.to_csv("SCENIC/Schiebinger/reprogramming_big_norm_adjacencies_1_692.csv")

In [12]:
reprogramming_big_norm_adjacencies_1 = pd.read_csv("SCENIC/Schiebinger/reprogramming_big_norm_adjacencies_1_692.csv")

In [16]:
modules_1 = list(modules_from_adjacencies(reprogramming_big_norm_adjacencies_1, exp_mtx))

/data/leuven/320/vsc32023/miniconda/envs/pySCENIC_2/lib/python3.6/site-packages/pyscenic/utils.py:138: RuntimeWarning: invalid value encountered in greater
  regulations = (rhos > rho_threshold).astype(int) - (rhos < -rho_threshold).astype(int)
/data/leuven/320/vsc32023/miniconda/envs/pySCENIC_2/lib/python3.6/site-packages/pyscenic/utils.py:138: RuntimeWarning: invalid value encountered in less
  regulations = (rhos > rho_threshold).astype(int) - (rhos < -rho_threshold).astype(int)


In [17]:
local_cluster = LocalCluster(n_workers=4, threads_per_worker=1)
client = Client(local_cluster)

In [18]:
# Calculate a list of enriched motifs and the corresponding target genes for all modules.
with ProgressBar():
    df = prune2df(dbs, modules_1, MOTIF_ANNOTATIONS_FNAME, client_or_address=client)

/data/leuven/320/vsc32023/miniconda/envs/pySCENIC_2/lib/python3.6/site-packages/distributed/worker.py:3101: UserWarning: Large object of size 1.09 MB detected in task graph: 
  (["('from-delayed-e0e12ee7ca9430b57ffa3ffe2d08d50f ... 0f', 18637)"],)
Consider scattering large objects ahead of time
with client.scatter to reduce scheduler burden and 
keep data on workers

    future = client.submit(func, big_data)    # bad

    big_future = client.scatter(big_data)     # good
    future = client.submit(func, big_future)  # good
  % (format_bytes(len(b)), s)


In [19]:
regulons_1 = df2regulons(df)

In [20]:
df.to_csv(MOTIFS_FNAME)

In [21]:
with open(REGULONS_FNAME, "wb") as f:
    pickle.dump(regulons_1, f)

In [22]:
auc_mtx = aucell(exp_mtx, regulons_1, num_workers=8)

In [23]:
auc_mtx.to_csv("SCENIC/Schiebinger/auc_reprogramming.big_embryo_norm_692.csv")